In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import sqlite3


In [2]:
def add_taxonomy(dataframe):
    result_values = {}
    for i in range(1, 16):
        codes_columns = f'Healthcare Provider Taxonomy Code_{i}'
        switch_columns = f'Healthcare Provider Primary Taxonomy Switch_{i}'
        
        # Check the condition and append values to the result list
        keys = dataframe['NPI'][dataframe[switch_columns]=='Y'].tolist()
        values = dataframe[codes_columns][dataframe[switch_columns]=='Y']
        for key, value in zip(keys,values):
            result_values[key] = value
            
    taxonomy_df = pd.DataFrame(list(result_values.items()), columns=['NPI', 'Taxonomy_Code'])
    nppes_merged = dataframe.merge(taxonomy_df, on = 'NPI', how = 'left')
    return nppes_merged

In [3]:
def Convert_strings_to_nan(value):
    try:
        return float(value)
    except ValueError:
        return np.nan

In [4]:
def fix_zipcode(series):
    series = pd.to_numeric(series)
    series = series.fillna(0)
    # create a new series with zipcodes
    new_series = series.apply(lambda x : str(int(x)).zfill(9) if len(str(int(x)))>5 else str(x).zfill(5))    
    # now i can split the strings at 5th item
    return new_series.str[:5]

In [5]:
columns_to_keep = ['NPI', 
                   'Entity Type Code', 
                   'Provider Organization Name (Legal Business Name)',
                   'Provider Last Name (Legal Name)',
                    'Provider First Name',
                    'Provider Middle Name',
                    'Provider Name Prefix Text',
                    'Provider Name Suffix Text',
                    'Provider Credential Text',       
                    'Provider First Line Business Practice Location Address',
                    'Provider Second Line Business Practice Location Address',
                    'Provider Business Practice Location Address City Name',
                    'Provider Business Practice Location Address State Name',
                    'Provider Business Practice Location Address Postal Code',
                    "Healthcare Provider Taxonomy Code_1",
                    "Healthcare Provider Primary Taxonomy Switch_1",
                    "Healthcare Provider Taxonomy Code_2",
                    "Healthcare Provider Primary Taxonomy Switch_2",
                    "Healthcare Provider Taxonomy Code_3",
                    "Healthcare Provider Primary Taxonomy Switch_3",
                    "Healthcare Provider Taxonomy Code_4",
                    "Healthcare Provider Primary Taxonomy Switch_4",
                    "Healthcare Provider Taxonomy Code_5",
                    "Healthcare Provider Primary Taxonomy Switch_5",
                    "Healthcare Provider Taxonomy Code_6",
                    "Healthcare Provider Primary Taxonomy Switch_6",
                    "Healthcare Provider Taxonomy Code_7",
                    "Healthcare Provider Primary Taxonomy Switch_7",
                    "Healthcare Provider Taxonomy Code_8",
                    "Healthcare Provider Primary Taxonomy Switch_8",
                    "Healthcare Provider Taxonomy Code_9",
                    "Healthcare Provider Primary Taxonomy Switch_9",
                    "Healthcare Provider Taxonomy Code_10",
                    "Healthcare Provider Primary Taxonomy Switch_10",
                    "Healthcare Provider Taxonomy Code_11",
                    "Healthcare Provider Primary Taxonomy Switch_11",
                    "Healthcare Provider Taxonomy Code_12",
                    "Healthcare Provider Primary Taxonomy Switch_12",
                    "Healthcare Provider Taxonomy Code_13",
                    "Healthcare Provider Primary Taxonomy Switch_13",
                    "Healthcare Provider Taxonomy Code_14",
                    "Healthcare Provider Primary Taxonomy Switch_14",
                    "Healthcare Provider Taxonomy Code_15",
                    "Healthcare Provider Primary Taxonomy Switch_15"
                    ]

In [6]:
cbsa = pd.read_csv('../data/ZIP_CBSA_122023.xlsx - Export Worksheet.csv')
taxonomy_code_classification =  pd.read_csv('../data/nucc_taxonomy_240.csv')

In [7]:
cbsa['zipcodes'] = fix_zipcode(cbsa['ZIP'])


In [8]:
# db = sqlite3.connect('../data/npi.sqlite')
# hop = []
# for chunk in pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000):
#      chunk = chunk[(chunk['transaction_count'] > 50) & (chunk['average_day_wait'] < 50)]
#      hop.append(chunk)
# hop_df = pd.concat(hop, ignore_index=True)
# hop_df.to_sql('hop_npi', db, if_exists='replace', index=False)
# db.execute('CREATE INDEX IF NOT EXISTS from_npi ON hop_npi(from_npi)')
# db.close()

In [9]:
# db = sqlite3.connect('../data/npi.sqlite')

# chunks = []
# for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', usecols=columns_to_keep, chunksize=10000):
#     chunk_taxonomy = add_taxonomy(chunk)
#     chunk_merged = pd.merge(left=chunk_taxonomy, right=taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), how='left', left_on='Taxonomy_Code', right_index=True)
#     chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
#     chunk_merged['zipcodes'] = fix_zipcode(chunk_merged['Provider Business Practice Location Address Postal Code'])
#     chunk_merged_cbsa = pd.merge(left=chunk_merged, right=cbsa[['zipcodes', 'CBSA']].set_index('zipcodes'), how='left', left_on='zipcodes', right_index=True)
#     chunk_merged_cbsa['CBSA'] = chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
#     chunks.append(chunk_merged_cbsa)

# npi = pd.concat(chunks, ignore_index=True)

# npi.to_sql('npi_nppes', db, if_exists = 'replace', index = False)

# db.execute('CREATE INDEX IF NOT EXISTS nppes_npi ON npi_nppes(NPI)')
# db.close()

In [10]:
#query confirming the amount of rows and columns are in the npi table
# db = sqlite3.connect('../data/npi.sqlite')
# query = "SELECT * FROM hop_npi" #WHERE patient_count > 1000"
# hop_teams = pd.read_sql(query, db)
# db.close()
# hop_teams

In [11]:
#query confirming the amount of rows and columns are in the nppes table
# db = sqlite3.connect('../data/npi.sqlite')
# query = "SELECT * FROM npi_nppes" #WHERE patient_count > 1000"
# nppes_teams = pd.read_sql(query, db)
# db.close()
# nppes_teams

In [26]:
# query = """
# SELECT "NPI", "Provider Organization Name (Legal Business Name)", "Classification", "from_npi", SUM("patient_count"), SUM("transaction_count")
# FROM npi_nppes AS nppes
# INNER JOIN hop_npi AS npi
# ON nppes.NPI = npi.from_npi
# WHERE "Classification" = 'Orthopaedic Surgery' OR "Classification" = 'Family Medicine' AND "Provider Business Practice Location Address City Name" LIKE "%NASHVILLE%"
# GROUP BY 1,2,3,4
# ORDER BY 5 DESC;
# """
# with sqlite3.connect('../data/npi.sqlite') as db:
#     two_classifications_2 = pd.read_sql(query, db)
    
# two_classifications_2

,NPI,Provider Organization Name (Legal Business Name),Classification,from_npi,"SUM(""patient_count"")","SUM(""transaction_count"")"
0,1144513375,"ST. VINCENT MEDICAL GROUP, INC.",Orthopaedic Surgery,1144513375,1244365,2012909
1,1376592964,EMERGEORTHO PA,Orthopaedic Surgery,1376592964,606427,1043588
2,1437382074,"COMMUNITY MEDICAL ASSOCIATES, INC.",Orthopaedic Surgery,1437382074,488356,737114
3,1497704217,"ORTHOCAROLINA, PA",Orthopaedic Surgery,1497704217,479617,770421
4,1205922432,ILLINOIS BONE AND JOINT INSTITUTE LLC,Orthopaedic Surgery,1205922432,462404,818764
...,...,...,...,...,...,...
25817,1972866457,"ALLEN ORTHOPEDICS,LLC",Orthopaedic Surgery,1972866457,18,56
25818,1629034137,None,Orthopaedic Surgery,1629034137,16,87
25819,1770945149,None,Family Medicine,1770945149,14,66
25820,1073886578,"INTEGRATED PHYSICAL MEDICINE, INC.",Orthopaedic Surgery,1073886578,13,109


In [20]:
# query = """
# SELECT *
# FROM hop_npi
# LIMIT 1;
# """
# with sqlite3.connect('../data/npi.sqlite') as db:
#     npi_columns = pd.read_sql(query, db)
    
# npi_columns

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923


In [4]:
# db = sqlite3.connect('../data/npi.sqlite')

# query = 'SELECT "Provider Organization Name (Legal Business Name)", "Provider Business Practice Location Address City Name", "NPI", "Classification", "Taxonomy_Code" FROM npi_nppes WHERE "Provider Business Practice Location Address City Name" LIKE "%NASHVILLE%"'

# Nashville_Practices = pd.read_sql(query, db)

# db.close()
#Nashville_Practices

In [ ]:
# #db = sqlite3.connect('data/npi.sqlite')
# for chunk in pd.read_csv('data/NPPES_Data_Dissemination_February_2024/npidata_pfile_20050523-20240211.csv', 
#                           usecols= columns_to_keep,
#                               chunksize = 10000):
#       chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
#     chunk_taxonomy = add_taxonomy(chunk)
#     chunk_merged = pd.merge(left = chunk_taxonomy, 
#                             right = taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), 
#                             how = 'left',
#                             left_on = 'Taxonomy_Code',
#                             right_index = True)
#     chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
#     chunk_merged['zipcodes']=fix_zipcode(
#         chunk_merged['Provider Business Practice Location Address Postal Code']
#         )
#     chunk_merged_cbsa = pd.merge(left = chunk_merged, 
#          right = cbsa[['zipcodes','CBSA']].set_index('zipcodes'), 
#          how = 'left',
#          left_on = 'zipcodes', 
#          right_index = True)
    
#     chunk_merged_cbsa['CBSA']= chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
    
#     chunk_merged_cbsa.to_sql('nppes', 
#                 db, 
#                 if_exists = 'append', 
#                 index = False)  

# #db.execute('CREATE INDEX NPI ON nppes(NPI)')
# db.close()

In [ ]:
# chunks = []

# for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', 
#                          usecols=columns_to_keep, 
#                          chunksize=10000):
#     #chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
#     chunk_taxonomy = add_taxonomy(chunk)
#     chunk_merged = pd.merge(left = chunk_taxonomy, right = taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), how ='left', left_on ='Taxonomy_Code', right_index = True)
#     chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
#     chunk_merged['zipcodes'] = fix_zipcode(chunk_merged['Provider Business Practice Location Address Postal Code'])
#     chunk_merged_cbsa = pd.merge(left = chunk_merged, right = cbsa[['zipcodes','CBSA']].set_index('zipcodes'), how ='left', left_on ='zipcodes', right_index = True)
#     chunk_merged_cbsa['CBSA'] = chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
#     chunks.append(chunk_merged_cbsa)

# # Concatenate all chunks into a single DataFrame
# npi = pd.concat(chunks, ignore_index=True)

In [ ]:
# hop = []
# chunks = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', 
#                               chunksize = 10000)
# for chunk in chunks:
#     chunk = chunk[chunk['transaction_count']>50]
#     chunk = chunk[chunk['average_day_wait']<50]
#     hop.append(chunk)
    
# hop = pd.concat(hop, ignore_index = True)

In [ ]:
# db = sqlite3.connect('../data/npi.sqlite')

# hop = []
# for chunk in pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000):
# chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
#     chunk = chunk[(chunk['transaction_count'] > 50) & (chunk['average_day_wait'] < 50)]
#     hop.append(chunk)

# hop_df = pd.concat(hop, ignore_index=True)

# hop_df.to_sql('hop_npi', db, if_exists='replace', index=False)

# db.execute('CREATE INDEX IF NOT EXISTS from_npi ON hop_npi(from_npi)')
# db.close()

In [ ]:
# db = sqlite3.connect('../data/npi.sqlite')

# chunks = []
# for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', usecols=columns_to_keep, chunksize=10000):
# chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
#     chunk_taxonomy = add_taxonomy(chunk)
#     chunk_merged = pd.merge(left=chunk_taxonomy, right=taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), how='left', left_on='Taxonomy_Code', right_index=True)
#     chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
#     chunk_merged['zipcodes'] = fix_zipcode(chunk_merged['Provider Business Practice Location Address Postal Code'])
#     chunk_merged_cbsa = pd.merge(left=chunk_merged, right=cbsa[['zipcodes', 'CBSA']].set_index('zipcodes'), how='left', left_on='zipcodes', right_index=True)
#     chunk_merged_cbsa['CBSA'] = chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
#     chunks.append(chunk_merged_cbsa)

# npi = pd.concat(chunks, ignore_index=True)

# npi.to_sql('npi_nppes', db, if_exists = 'replace', index = False)

# db.execute('CREATE INDEX IF NOT EXISTS NPI ON npi_nppes(NPI)')
# db.close()

In [ ]:
#What are the major Nashville Hospitals?
    #Vanderbilt
    #Centennial
    #St. Thomas (midtown, west)
    #Nashville General
    #VA